<a href="https://colab.research.google.com/github/hyun-hyang/ResumEmphasizer/blob/sub/resume.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spacy_transformers
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 2.1 MB/s 
     |████████████████████████████████| 4.4 MB 30.0 MB/s 
     |████████████████████████████████| 1.1 MB 57.9 MB/s 
     |████████████████████████████████| 101 kB 13.2 MB/s 
     |████████████████████████████████| 596 kB 71.3 MB/s 
     |████████████████████████████████| 6.6 MB 58.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [ ]:
spacy.__version__

'3.4.1'

In [ ]:
!nvidia-smi

Sat Aug  6 19:56:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W /  70W |    312MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [27]:
%cd /content/ResumEmphasizer
!git pull

/content/ResumEmphasizer
Already up to date.


In [ ]:
cv_data = json.load(open('/content/ResumEmphasizer/Data/resumeconverted.json','r'))

In [ ]:
len(cv_data)

49

In [ ]:
!python -m spacy init fill-config base_config.cfg config.cfg

Usage: python -m spacy init fill-config 
           [OPTIONS] BASE_PATH [OUTPUT_FILE]
Try 'python -m spacy init fill-config --help' for help.

Error: Invalid value for 'BASE_PATH': File 'base_config.cfg' does not exist.


In [ ]:
!python -m spacy init fill-config /content/ResumEmphasizer/Data/base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
cv_data

In [ ]:
#from re import T
def get_spacy_doc(file, data):
  nlp = spacy.blank('en')
  db = DocBin()

  for item in data:
    text = item['text']
    doc = nlp.make_doc(text)
    annot = item['label']

    ents = []
    entity_indices = []

    for start, end, label in annot:
      skip_entity = False
      for idx in range(start, end):
        if idx in entity_indices:
          break
      if skip_entity == True:
        continue

      entity_indices = entity_indices + list(range(start, end))

      try:
        span = doc.char_span(start, end , label=label, alignment_mode = 'strict')
      except:
        continue

      if span is None:
        err_data = str([start, end]) + "    " + str(text) + '\n'
        file.write(err_data)

      else:
        ents.append(span)
      
    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass

    return db
      

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data , test_size=0.3)

In [ ]:
len(train), len(test)

(34, 15)

In [ ]:
file = open('error.txt','w')

db = get_spacy_doc(file, train)
db.to_disk('train_data.spacy')

db = get_spacy_doc(file,train)
db.to_disk('test_data.spacy')

file.close()

In [ ]:
len(db.tokens)

1

In [16]:
!python -m spacy train /content/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-08-06 19:57:15,650] [INFO] Set up nlp object from config
[2022-08-06 19:57:15,661] [INFO] Pipeline: ['transformer', 'ner']
[2022-08-06 19:57:15,665] [INFO] Created vocabulary
[2022-08-06 19:57:15,666] [INFO] Finished initializing nlp object
Downloading: 100% 481/481 [00:00<00:00, 553kB/s]
Downloading: 100% 878k/878k [00:00<00:00, 1.75MB/s]
Downloading: 100% 446k/446k [00:00<00:00, 1.07MB/s]
Downloading: 100% 1.29M/1.29M [00:00<00:00, 3.10MB/s]
Downloading: 100% 478M/478M [00:07<00:00, 67.7MB/s]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a mod

Model Test

In [17]:
nlp = spacy.load('/content/output/model-best')


In [18]:
doc = nlp('my name is Leo. I worked at Microsoft. I have 10 years of experience')
for ent in doc.ents:
  print(ent.text, "    ->>>>  ", ent.label_)

my     ->>>>   work experience
name     ->>>>   work experience
is     ->>>>   work experience
Leo     ->>>>   work experience
.     ->>>>   work experience
I     ->>>>   work experience
worked     ->>>>   work experience
at     ->>>>   work experience
Microsoft     ->>>>   work experience
.     ->>>>   work experience
I     ->>>>   work experience
have     ->>>>   work experience
10     ->>>>   work experience
years     ->>>>   work experience
of     ->>>>   work experience
experience     ->>>>   work experience


In [19]:
!pip install PyMuPDF

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.8 MB 14.6 MB/s 


In [20]:
import sys, fitz

In [28]:
fname = '/content/ResumEmphasizer/Data/cv250.pdf'
doc = fitz.open(fname)

In [22]:
#doc = [page.getText() for page in doc]

In [29]:
text = " "
for page in doc:
  text = text + str(page.get_text())

In [30]:
text = text.strip()

In [31]:
text = ' '.join(text.split())

In [32]:
doc = nlp(text)
for ent in doc.ents:
  print(ent.text, "    ->>>>  ", ent.label_)

Parveen     ->>>>   work experience
Kaur     ->>>>   work experience
Noida     ->>>>   work experience
,     ->>>>   work experience
Uttar     ->>>>   work experience
Pradesh     ->>>>   work experience
Work     ->>>>   work experience
Experience     ->>>>   work experience
Java     ->>>>   work experience
Developer     ->>>>   work experience
Kineticx     ->>>>   work experience
Tech     ->>>>   work experience
Solutions     ->>>>   work experience
-     ->>>>   work experience
Noida     ->>>>   work experience
,     ->>>>   work experience
Uttar     ->>>>   work experience
Pradesh     ->>>>   work experience
April     ->>>>   work experience
2016     ->>>>   work experience
to     ->>>>   work experience
Present     ->>>>   work experience
Designation     ->>>>   work experience
:     ->>>>   work experience
Java     ->>>>   work experience
Developer     ->>>>   work experience
PROJECT     ->>>>   work experience
NAME     ->>>>   work experience
:     ->>>>   work experience
Kx     -